<a href="https://colab.research.google.com/github/ffedox/pbr/blob/main/text_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup

## 1.1 Libraries

In [4]:
from nltk.tokenize import word_tokenize
import nltk

import pandas as pd
import os
import re

In [ ]:
# In case there are problems with NLTK ->
# !pip install -U nltk 

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 2. Merging the extracted parallel corpora

`path = os.getcwd()` sets the variable `path` to the current working directory using the `os.getcwd()` method.


`files = os.listdir(path)` sets the variable `files` to a list of all the files and directories in the current working directory, which is stored in the variable `path`.

In [ ]:
path = os.getcwd()
files = os.listdir(path)
files

['.config',
 'parallel_corpus_vg_en_it_3.xlsx',
 'drive',
 'parallel_corpus_vg_en_it_5.xlsx',
 'parallel_corpus_vg_en_it_19.xlsx',
 'parallel_corpus_vg_en_it_12.xlsx',
 'parallel_corpus_vg_en_it_13.xlsx',
 'parallel_corpus_vg_en_it_18.xlsx',
 'parallel_corpus_vg_en_it_10.xlsx',
 'parallel_corpus_vg_en_it_16.xlsx',
 'parallel_corpus_vg_en_it_15.xlsx',
 'parallel_corpus_vg_en_it_8.xlsx',
 'parallel_corpus_vg_en_it_20.xlsx',
 'parallel_corpus_vg_en_it_9.xlsx',
 'parallel_corpus_vg_en_it_14.xlsx',
 'parallel_corpus_vg_en_it_2.xlsx',
 'parallel_corpus_vg_en_it_11.xlsx',
 'parallel_corpus_vg_en_it_1.xlsx',
 'parallel_corpus_vg_en_it_7.xlsx',
 'parallel_corpus_vg_en_it_21.xlsx',
 'parallel_corpus_vg_en_it_4.xlsx',
 'parallel_corpus_vg_en_it_17.xlsx',
 'parallel_corpus_vg_en_it_6.xlsx',
 'sample_data']

This list comprehension creates a new list `files_xlsx` that contains all the elements from the `files` list that end with `.xlsx`.

In [ ]:
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
files_xlsx

['parallel_corpus_vg_en_it_3.xlsx',
 'parallel_corpus_vg_en_it_5.xlsx',
 'parallel_corpus_vg_en_it_19.xlsx',
 'parallel_corpus_vg_en_it_12.xlsx',
 'parallel_corpus_vg_en_it_13.xlsx',
 'parallel_corpus_vg_en_it_18.xlsx',
 'parallel_corpus_vg_en_it_10.xlsx',
 'parallel_corpus_vg_en_it_16.xlsx',
 'parallel_corpus_vg_en_it_15.xlsx',
 'parallel_corpus_vg_en_it_8.xlsx',
 'parallel_corpus_vg_en_it_20.xlsx',
 'parallel_corpus_vg_en_it_9.xlsx',
 'parallel_corpus_vg_en_it_14.xlsx',
 'parallel_corpus_vg_en_it_2.xlsx',
 'parallel_corpus_vg_en_it_11.xlsx',
 'parallel_corpus_vg_en_it_1.xlsx',
 'parallel_corpus_vg_en_it_7.xlsx',
 'parallel_corpus_vg_en_it_21.xlsx',
 'parallel_corpus_vg_en_it_4.xlsx',
 'parallel_corpus_vg_en_it_17.xlsx',
 'parallel_corpus_vg_en_it_6.xlsx']

Creating an empty DataFrame using the `pandas` library.

In [ ]:
df = pd.DataFrame()

Looping over all the elements in the `files_xlsx` list and combining their data into a single DataFrame.

In [ ]:
for f in files_xlsx:
    data = pd.read_excel(f, index_col=0)
    df = df.append(data)

# 3. Preprocessing

Using the `drop_duplicates()` method from the `pandas` library to remove duplicate rows.

In [ ]:
df = df.drop_duplicates()

Replacing all newline characters in the 'en' and 'it' columns of the DataFrame `df` with a whitespace.

In [ ]:
df['en'] = df['en'].apply(lambda x: re.sub('\n', ' ', str(x))) 
df['it'] = df['it'].apply(lambda x: re.sub('\n', ' ', str(x)))

Removing all special characters except punctuation and accented letters from the 'en' and 'it' columns of the DataFrame `df`.

In [ ]:
df['en'] = df['en'].apply(lambda x: re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ\/\-.,;:"\']+', '', str(x))) 
df['it'] = df['it'].apply(lambda x: re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ\/\-.,;:"\']+', '', str(x)))

Tokenizing the text and counting the tokens of the 'en' and 'it' columns of the DataFrame `df`.

In [ ]:
df['tokenized_text_en'] = df['en'].apply(word_tokenize) 
df['token_count_en'] = df['tokenized_text_en'].apply(lambda x: len(x))

df['tokenized_text_it'] = df['it'].apply(word_tokenize) 
df['token_count_it'] = df['tokenized_text_it'].apply(lambda x: len(x))

Filtering the DataFrame `df` to only keep the rows where the number of tokens in the 'token_count_en' column is greater than or equal to 6.

In [ ]:
df = df[(df['token_count_en'] >= 6)]

Creating a new column named 'difference', which represents the absolute difference between the number of tokens in the 'token_count_en' and 'token_count_it' columns, and then filtering the DataFrame to only keep the rows where the difference is less than 10.

In [ ]:
df['difference'] = df["token_count_en"] - df["token_count_it"]
df['difference'] = df['difference'].abs()
df = df[(df['difference'] < 10)]

Resetting the index of the DataFrame. The `drop` argument is set to `True`, which means that the old index values are discarded and not added as a new column to the DataFrame.

In [ ]:
df = df.reset_index(drop=True)

# 4. Counting the tokens

Calculating the sum of the values in the 'token_count_en' and 'token_count_it' columns of the DataFrame and assigning the results to the variables `total_tokens_en` and `total_tokens_it`, respectively.

In [ ]:
total_tokens_en = df['token_count_en'].sum()
total_tokens_it = df['token_count_it'].sum()

In [ ]:
print(total_tokens_en)

259354


In [ ]:
print(total_tokens_it)

265349


# 5. Exporting to Excel

Writing the content of the DataFrame to an Excel spreadsheet named 'parallel_corpus.xlsx'.

In [ ]:
df.to_excel('parallel_corpus.xlsx')  